<a href="https://colab.research.google.com/github/ipediz/BachehayekhafaneTMU/blob/main/Panel_OLS_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This code was written with the aid of ChatGPT**

In [ ]:

import numpy as np
import pandas as pd
import statsmodels.api as sm
import sklearn.linear_model as lm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from linearmodels.panel import PanelOLS
import warnings
# ignore potential convergence warnings; for some small penalty levels,
# tried out, optimization might not converge
warnings.simplefilter('ignore')

In [ ]:
Data = pd.read_csv("The_new_all_state_replication_data.csv")
Data['interaction'] = Data['interaction'] / 100
Data['percent_over_60'] = Data['percent_over_60'] / 100
# setting 'county_code' as index and 'new_timeperiod' as time



In [ ]:
from tabulate import tabulate
#dropping non-relative variables for summary statistics table
columns_to_drop = [
    'relative_time_m8andbeyond', 'relative_time_m7', 'relative_time_m6', 'relative_time_m5',
    'relative_time_m4', 'relative_time_m3', 'relative_time_m2', 'relative_time_m1', 'relative_time_0',
    'relative_time_p1', 'relative_time_p2', 'relative_time_p3', 'relative_time_p4',
    'relative_time_p5', 'relative_time_p6', 'relative_time_p7', 'relative_time_p8andbeyond'
]
Data = Data.set_index(['county_code', 'new_timeperiod'])
# Create a new dataframe without the specified columns
Data1 = Data.drop(columns=columns_to_drop)

In [ ]:
# Generate summary statistics for all numerical columns

Data1_Summary_Statistics = Data1.describe()

# Convert the summary statistics DataFrame to a format that tabulate can handle
table = tabulate(Data1_Summary_Statistics, headers=Data1_Summary_Statistics.columns, tablefmt='fancy_grid')

# Print the beautiful table
print(table)

╒═══════╤═══════════════╤══════════════════╤══════════════════════════╤═════════════════════════╤══════════════════╤═════════════════════════╤════════════════════════╤════════════════════╤═══════════════╤═════════════════════════╤══════════════╤═══════════════╤════════════════════════════════════╤═════════════════════════════════╤═══════════════════╤═══════════════╕
│       │   county_code │   new_timeperiod │   lending_club_available │   bankruptcy_per_capita │   log_bankruptcy │   population_estimation │   employed_individuals │   monthly_earnings │   labor_force │   medianhouseholdincome │   cem_strata │   cem_weights │   nonbusiness_bankruptcy_percapita │   business_bankruptcy_percapita │   percent_over_60 │   interaction │
╞═══════╪═══════════════╪══════════════════╪══════════════════════════╪═════════════════════════╪══════════════════╪═════════════════════════╪════════════════════════╪════════════════════╪═══════════════╪═════════════════════════╪══════════════╪═══════════════╪═

In [ ]:
Data1 = Data1.dropna()
train, test = train_test_split(Data1, test_size=0.20, random_state=123)
train = train.set_index(['county_code', 'new_timeperiod'])

Regression Model


In [ ]:
model_base = 'bankruptcy_per_capita  ~ lending_club_available  + interaction  + population_estimation + employed_individuals  + medianhouseholdincome + percent_over_60  '
base = PanelOLS.from_formula(model_base, data=train, weights=train['cem_weights'])
results_base = base.fit(cov_type='clustered', cluster_entity=True)
print(results_base.summary)

                            PanelOLS Estimation Summary                            
Dep. Variable:     bankruptcy_per_capita   R-squared:                        0.7166
Estimator:                      PanelOLS   R-squared (Between):              0.8179
No. Observations:                  51488   R-squared (Within):              -0.0472
Date:                   Fri, Oct 18 2024   R-squared (Overall):              0.7166
Time:                           18:43:30   Log-likelihood                -2.811e+04
Cov. Estimator:                Clustered                                           
                                           F-statistic:                    2.17e+04
Entities:                           2302   P-value                           0.0000
Avg Obs:                          22.367   Distribution:                 F(6,51482)
Min Obs:                          8.0000                                           
Max Obs:                          28.000   F-statistic (robust):            

In [ ]:

rsquared_base = results_base.rsquared
rsquared_adj_base = results_base.rsquared_within  # For fixed effects models

# Calculate MSE manually
mse_base = np.mean(results_base.resids**2)

# Print the results
print(f'Rsquared={rsquared_base:.4f}')
print(f'Rsquared_within_adjusted={rsquared_adj_base:.4f}')
print(f'MSE={mse_base:.4f}')


Rsquared=0.7166
Rsquared_within_adjusted=-0.0472
MSE=0.1738
